In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
import glob
from importlib import reload
import extract_features
import os
import numpy as np
import json

json_path_project = 'S:\\AG\\AG-Bewegungsstoerungen-II\\LFP\\PROJECTS\BATTERY\\'

### 1. Importing all Json files from Directory, and extracting the features

In [ ]:
json_path_Subject = 'Sub047\\Ward'

all_json_files = glob.glob(os.path.join(json_path_project, json_path_Subject, '*.json'))
filtered_files = [file for file in all_json_files if not os.path.basename(file).startswith('MetaTable') and not os.path.basename(file).startswith('Sub')]
filtered_files

In [ ]:
'''with open(os.path.join(json_path, 'Report_Json_Session_Report_20200605T115803.json')) as file:
            # Load the JSON data
            data = json.load(file)
'''

In [ ]:
reload(extract_features)
SUBID = json_path_Subject.split('\\', 1)[0]
json_path = os.path.join(json_path_project, json_path_Subject)
extract_features.extract_MetaTable(json_path_Subject, json_path, SUBID, filtered_files)

### 2. Merging all Dataframes together for each subject

In [ ]:
mySub = SUBID
All_MetaTables_ThisSub_path = os.path.join(json_path_project, mySub)

# Initialize an empty list to store the DataFrames
dfs = []

# Iterate through the files in the directory
for filename in os.listdir(All_MetaTables_ThisSub_path):
    if filename.endswith(".json"):
        # Construct the full file path
        file_path = os.path.join(All_MetaTables_ThisSub_path, filename)
        
        # Read the JSON file and append the DataFrame to the list
        df = pd.read_json(file_path)
        dfs.append(df)

# Concatenate all DataFrames into a single DataFrame
All_MetaTables_ThisSub_Combined = pd.concat(dfs, ignore_index=True)
conversion_factor = 24 * 60 * 60
All_MetaTables_ThisSub_Combined['AccumulatedTherapyOnTimeSinceImplant_Initial_Days'] = All_MetaTables_ThisSub_Combined['AccumulatedTherapyOnTimeSinceImplant_Initial'] / conversion_factor
All_MetaTables_ThisSub_Combined.sort_values('AccumulatedTherapyOnTimeSinceImplant_Initial', inplace=True)
All_MetaTables_ThisSub_Combined.tail()

In [ ]:
'''All_MetaTables_ThisSub_Combined = pd.read_json(os.path.join(
    'C:\\Users\\mathiopv\\OneDrive - Charité - Universitätsmedizin Berlin\\BATTERY_LIFE\\results\\MetaTable_Alls',
    'Sub005_MetaTable_All.json'
))

All_MetaTables_ThisSub_Combined.head()'''

#### 2.1 Curating the Merged Df: Duplicates

In [ ]:
duplicates = All_MetaTables_ThisSub_Combined.duplicated(subset=['SessionDate', 'AccumulatedTherapyOnTimeSinceImplant_Initial'], keep=False)
duplicate_indices = duplicates[duplicates].index.tolist()
duplicate_rows = All_MetaTables_ThisSub_Combined[duplicates]

duplicate_rows

In [ ]:
All_MetaTables_ThisSub_Combined.drop_duplicates(subset=['SessionDate', 'AccumulatedTherapyOnTimeSinceImplant_Initial'], keep='first', inplace=True)
All_MetaTables_ThisSub_Combined

In [ ]:
'''# Get the last row of 'BatteryPercentage' for each unique 'Con_Reason' value
last_battery_percentages = All_MetaTables_ThisSub_Combined.groupby('Con_Reason')['BatteryPercentage'].last()

# Calculate the sum of 'overallSensingDurSec' for each unique 'Con_Reason' value
sum_sensing_duration = All_MetaTables_ThisSub_Combined.groupby('Con_Reason')['overallSensingDurSec'].sum()/60
sum_telemetry_duration = All_MetaTables_ThisSub_Combined.groupby('Con_Reason')['telemetry_durationSec'].sum()/60

# Iterate through unique 'Con_Reason' values
for con_reason in All_MetaTables_ThisSub_Combined['Con_Reason'].unique():
    print(f"Last Battery Percentage for {con_reason}: {last_battery_percentages[con_reason]}")
    print(f"Sum of overallSensingDurSec for {con_reason}: {sum_sensing_duration[con_reason]}")
    print(f"Sum of telemetry_durationSec for {con_reason}: {sum_telemetry_duration[con_reason]}")
    print()'''

#### 2.2 Curating the Merged Df: Determine Follow-Up

In [ ]:
'''%matplotlib qt
plt.scatter(All_MetaTables_ThisSub_Combined['AccumulatedTherapyOnTimeSinceImplant_Initial_Days'], All_MetaTables_ThisSub_Combined['BatteryPercentage'])
plt.show()'''

In [ ]:
'''postOp_cutoff = 50
fu3m_cutoff = 120
fu12_cutoff = 360

All_MetaTables_ThisSub_Combined.loc[
    All_MetaTables_ThisSub_Combined['AccumulatedTherapyOnTimeSinceImplant_Initial_Days'] < postOp_cutoff, 'FollowUp'] = 'FU0M'

All_MetaTables_ThisSub_Combined.loc[
    (All_MetaTables_ThisSub_Combined['AccumulatedTherapyOnTimeSinceImplant_Initial_Days'] > postOp_cutoff) &
    (All_MetaTables_ThisSub_Combined['AccumulatedTherapyOnTimeSinceImplant_Initial_Days'] < fu3m_cutoff), 'FollowUp'] = 'FU3M'

All_MetaTables_ThisSub_Combined.loc[
    All_MetaTables_ThisSub_Combined['AccumulatedTherapyOnTimeSinceImplant_Initial_Days'] > fu3m_cutoff, 'FollowUp'] = 'FU12M'

All_MetaTables_ThisSub_Combined.loc[
All_MetaTables_ThisSub_Combined['AccumulatedTherapyOnTimeSinceImplant_Initial_Days'] > fu12_cutoff, 'FollowUp'] = 'LongTerm'

All_MetaTables_ThisSub_Combined.loc[
    All_MetaTables_ThisSub_Combined['Con_Reason'] == 'Beelitz', 'FollowUp'] = 'Beelitz'

# Calculate mean of 'overallSensingDurSec' for rows where 'Con_Reason' is 'Ambulant'
mean_dur = All_MetaTables_ThisSub_Combined.loc[All_MetaTables_ThisSub_Combined['Con_Reason'] == 'Ambulant', 'overallSensingDurSec'].mean()
mean_dur1 = All_MetaTables_ThisSub_Combined.loc[All_MetaTables_ThisSub_Combined['Con_Reason'] == 'TAmbulant', 'overallSensingDurSec'].mean()

# Update 'Con_Reason' column based on the mean value
All_MetaTables_ThisSub_Combined.loc[All_MetaTables_ThisSub_Combined['Con_Reason'] == 'Ambulant', 'Con_Reason'] = \
    All_MetaTables_ThisSub_Combined.loc[All_MetaTables_ThisSub_Combined['Con_Reason'] == 'Ambulant'].apply(lambda row: 'Forschung' if row['overallSensingDurSec'] >= 10 else 'Ward', axis=1)

All_MetaTables_ThisSub_Combined.loc[All_MetaTables_ThisSub_Combined['Con_Reason'] == 'TAmbulant', 'Con_Reason'] = \
    All_MetaTables_ThisSub_Combined.loc[All_MetaTables_ThisSub_Combined['Con_Reason'] == 'TAmbulant'].apply(lambda row: 'Forschung' if row['overallSensingDurSec'] >= 10 else 'Ward', axis=1)


#All_MetaTables_ThisSub_Combined['Con_Reason'] = All_MetaTables_ThisSub_Combined['Con_Reason'].apply(lambda x: 'Forschung' if x == 'True' else 'Ward')
All_MetaTables_ThisSub_Combined.head(20)'''

In [ ]:
#All_MetaTables_ThisSub_Combined.at[35, 'telemetry_durationSec'] = np.nan

'''[datetime.strptime(t, "%Y-%m-%dT%H:%M:%SZ") for t in All_MetaTables_ThisSub_Combined['SessionDate']]
arr_timestamps

import 3m timestamp
i_3mfu = np.argmin(abs(arr_timestamps - 3m_time))'''

## If all is Ok, save Dataframe Now!

In [ ]:
MetaTable_Results_Path = 'C:\\Users\\mathiopv\\OneDrive - Charité - Universitätsmedizin Berlin\\BATTERY_LIFE\\results\\MetaTable_Alls'

All_MetaTables_ThisSub_Combined.to_json(os.path.join(
    MetaTable_Results_Path,
    f'{mySub}_MetaTable_All.json'), 
    orient='records')

In [ ]:
#### Convert the json files to excels in order to fix FU

MetaTable_Results_Path = 'C:\\Users\\mathiopv\\OneDrive - Charité - Universitätsmedizin Berlin\\BATTERY_LIFE\\results\\MetaTable_Alls'

json_files = [f for f in os.listdir(MetaTable_Results_Path) if f.endswith('.json')]

for json_file in json_files:
    # Read the JSON file into a dataframe
    json_path = os.path.join(MetaTable_Results_Path, json_file)
    df = pd.read_json(json_path)
    
    # Extract the filename without the extension
    file_name_without_extension, _ = os.path.splitext(json_file)
    
    # Create the Excel file path
    excel_file = os.path.join(MetaTable_Results_Path, file_name_without_extension + '.xlsx')
    
    # Save the dataframe as an Excel file
    df.to_excel(excel_file, index=False)
    print(f"Converted {json_file} to {file_name_without_extension}.xlsx")


### 3. Make average features for each follow-up and save them + Figures

In [ ]:
path = 'C:\\Users\\mathiopv\\OneDrive - Charité - Universitätsmedizin Berlin\\BATTERY_LIFE\\results\\MetaTable_Alls'
results_path_object = 'C:\\Users\\mathiopv\\OneDrive - Charité - Universitätsmedizin Berlin\\BATTERY_LIFE\\results\\Avg_Features'
figures_path_object = 'C:\\Users\\mathiopv\\OneDrive - Charité - Universitätsmedizin Berlin\\BATTERY_LIFE\\figures\\Overview\\'


file_list = []

# Loop through all files in the directory
for filename in os.listdir(path):
    if filename.endswith('Sub024_MetaTable_All.xlsx') or filename.endswith('Sub024_MetaTable_All.xlsx'):  # Filter Excel files
        file_list.append(filename)

for filename in file_list:

    All_MetaTables_ThisSub_Combined = pd.read_excel(os.path.join(path, filename))

    sub_id = filename[0:6]

    sums = pd.DataFrame(columns=['TimePoint', 'Telemetry_AllSec', 'TelemDurSumSecRes','TelemDurSumSecWard','SensDurSumSec', 'FirstBatVal', 'LastBatVal'])
    # Get the unique time points in the order they appear
    time_points_sorted = All_MetaTables_ThisSub_Combined['FollowUp'].unique()

    # Sort the time points based on their first occurrence in the DataFrame
    #time_points_sorted = sorted(time_points, key=lambda x: np.where(All_MetaTables_ThisSub_Combined['FollowUp'] == x)[0][0])

    # Get the corresponding y-values (last battery values)
    first_battery_values = All_MetaTables_ThisSub_Combined.groupby('FollowUp')['BatteryPercentage'].first()
    last_battery_values = All_MetaTables_ThisSub_Combined.groupby('FollowUp')['BatteryPercentage'].last()
    #y_values = [last_battery_values[tp] if tp in last_battery_values.index else 0 for tp in time_points_sorted]


    for tp in time_points_sorted:
        subset = All_MetaTables_ThisSub_Combined[All_MetaTables_ThisSub_Combined['FollowUp'] == tp]
        telemetry_all = subset['telemetry_durationSec'].sum()
        telemetry_sumRes = subset.loc[subset['Con_Reason'] == 'Forschung', 'telemetry_durationSec'].sum()
        telemetry_sumWard = subset.loc[(subset['Con_Reason'] == 'Ward') | (subset['Con_Reason'] == 'Beelitz'), 'telemetry_durationSec'].sum()
        sensing_sum = subset['overallSensingDurSec'].sum()
        first_battery_value = first_battery_values[tp] if tp in first_battery_values.index else None
        last_battery_value = last_battery_values[tp] if tp in last_battery_values.index else None
        
        sums = sums.append({
            'TimePoint': tp,
            'Telemetry_AllSec': telemetry_all,
            'TelemDurSumSecRes': telemetry_sumRes,
            'TelemDurSumSecWard': telemetry_sumWard,
            'SensDurSumSec': sensing_sum,
            'FirstBatVal': first_battery_value,
            'LastBatVal': last_battery_value
        }, ignore_index=True)

    sums.to_csv(os.path.join(
    results_path_object,
    f'{sub_id}_AvgFeatures.csv'
    ))

    ####### PLOT IT #######
    tp_of_int = ['FU0M','Beelitz','FU3M','FU12M']

    # Filter the DataFrame to include only the specified follow-ups
    filtered_df = sums [sums ['TimePoint'].isin(tp_of_int)]

    %matplotlib inline

    fig, axs = plt.subplots(1,4, figsize=(18,4))

    axs[0].bar(filtered_df['TimePoint'], filtered_df['Telemetry_AllSec']/60)
    axs[0].bar(filtered_df['TimePoint'], filtered_df['TelemDurSumSecRes']/60, label = 'Research')
    axs[0].set_ylabel('Duration [min]')
    axs[0].set_title('Telemetry Duration')
    axs[0].legend()

    axs[1].bar(filtered_df['TimePoint'], filtered_df['SensDurSumSec']/60)
    axs[1].set_ylabel('Duration [min]')
    axs[1].set_title('Sensing Duration')

    subset = filtered_df[filtered_df['TimePoint'].isin(['FU0M', 'Beelitz', 'FU3M', 'FU12M'])]
    axs[2].bar(subset['TimePoint'], subset['TelemDurSumSecWard']/60)
    axs[2].set_ylabel('Duration [min]')
    axs[2].set_title('Ward Telemetry')

    axs[3].bar(filtered_df['TimePoint'], filtered_df['LastBatVal'])
    axs[3].set_ylabel('IPG Battery [%]')
    axs[3].set_title('Battery')

    plt.suptitle(sub_id)

    AvgValues_ThisSub_FigName = f'{sub_id}_Overview'

    fig.savefig(os.path.join(
        figures_path_object, AvgValues_ThisSub_FigName),
        dpi = 150)


### 6. Find first 12mfu connection

In [ ]:
path = 'C:\\Users\\mathiopv\\OneDrive - Charité - Universitätsmedizin Berlin\\BATTERY_LIFE\\results\\MetaTable_Alls'

jsons_12mfu = pd.DataFrame(columns=['SubID', 'Con_reason','json_fileName'])

for filename in os.listdir(path):
    if filename.endswith('.xlsx'):  # Filter Excel files
    
        thisMetaTable = pd.read_excel(os.path.join(path, filename))
        
        sub_id = filename[0:6]
        fu12m_subset = thisMetaTable[thisMetaTable['FollowUp']=='FU12M']
        fu12m_subset = fu12m_subset.reset_index(drop=True)
        json_filename = fu12m_subset.loc[0,'json_fileName']
        con_reason = fu12m_subset.loc[0,'Con_Reason']
        
        jsons_12mfu = jsons_12mfu.append({
                'SubID': sub_id,
                'Con_reason': con_reason,
                'json_fileName': json_filename
            }, ignore_index=True)

In [ ]:
jsons_12mfu.to_excel(
    os.path.join(
        path,'Jsons_12mfu_StimPars_ToExtract.xlsx'
    )
)